In [1]:
# Import our dependencies
import os 
import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

In [2]:
#Import the dataset 
companies_df = pd.read_csv('Unicorn_companies.csv')
companies_df.head()

,Company,Valuation ($B),Date Joined,Country,City,Industry,Select Inverstors,Founded Year,Total Raised,Financial Stage,Investors Count,Deal Terms,Portfolio Exits
0,Bytedance,$140,4/7/2017,China,Beijing,Artificial intelligence,"Sequoia Capital China, SIG Asia Investments, S...",2012,$7.44B,IPO,28,8,5
1,SpaceX,$100.3,12/1/2012,United States,Hawthorne,Other,"Founders Fund, Draper Fisher Jurvetson, Rothen...",2002,$6.874B,None,29,12,None
2,Stripe,$95,1/23/2014,United States,San Francisco,Fintech,"Khosla Ventures, LowercaseCapital, capitalG",2010,$2.901B,Asset,39,12,1
3,Klarna,$45.6,12/12/2011,Sweden,Stockholm,Fintech,"Institutional Venture Partners, Sequoia Capita...",2005,$3.472B,Acquired,56,13,1
4,Epic Games,$42,10/26/2018,United States,Cary,Other,"Tencent Holdings, KKR, Smash Ventures",1991,$4.377B,Acquired,25,5,2


In [3]:

#Generate out categorical variable list 
companies_cat = companies_df.dtypes[companies_df.dtypes=="object"].index.tolist()

In [4]:
#Check for unique values
companies_df[companies_cat].nunique()

Company              1035
Valuation ($B)        200
Date Joined           623
Country                46
City                  256
Industry               33
Select Inverstors    1006
Founded Year           37
Total Raised          914
Financial Stage        10
Investors Count        54
Deal Terms             16
Portfolio Exits         5
dtype: int64

In [5]:
# companies_df['Valuation ($B)'].dtypes
companies_df['Valuation ($B)']= companies_df['Valuation ($B)'].str.replace('$','')
companies_df['Valuation ($B)'] = companies_df['Valuation ($B)'].astype(float)
# companies_df.dtypes 

# sorted_df = companies_df.sort_values(by= 'Valuation ($B)',ascending = False, inplace = True)
companies_df.head()

/Users/Biggaharry/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


,Company,Valuation ($B),Date Joined,Country,City,Industry,Select Inverstors,Founded Year,Total Raised,Financial Stage,Investors Count,Deal Terms,Portfolio Exits
0,Bytedance,140.0,4/7/2017,China,Beijing,Artificial intelligence,"Sequoia Capital China, SIG Asia Investments, S...",2012,$7.44B,IPO,28,8,5
1,SpaceX,100.3,12/1/2012,United States,Hawthorne,Other,"Founders Fund, Draper Fisher Jurvetson, Rothen...",2002,$6.874B,None,29,12,None
2,Stripe,95.0,1/23/2014,United States,San Francisco,Fintech,"Khosla Ventures, LowercaseCapital, capitalG",2010,$2.901B,Asset,39,12,1
3,Klarna,45.6,12/12/2011,Sweden,Stockholm,Fintech,"Institutional Venture Partners, Sequoia Capita...",2005,$3.472B,Acquired,56,13,1
4,Epic Games,42.0,10/26/2018,United States,Cary,Other,"Tencent Holdings, KKR, Smash Ventures",1991,$4.377B,Acquired,25,5,2


In [16]:
#Check for null values
import numpy as np
companies_df["Valuation ($B)"].isnull()


0       False
1       False
2       False
3       False
4       False
        ...  
1032    False
1033    False
1034    False
1035    False
1036    False
Name: Valuation ($B), Length: 1037, dtype: bool

In [17]:
val_counts = companies_df['Valuation ($B)'].value_counts()
val_counts

1.00    244
2.00     67
1.50     53
1.20     40
1.10     34
       ... 
7.10      1
7.30      1
7.75      1
7.60      1
3.44      1
Name: Valuation ($B), Length: 200, dtype: int64

In [18]:
companies_df['Total Raised'].values

array(['$7.44B', '$6.874B', '$2.901B', ..., '$449.72M', '$525.5M',
       '$604.5M'], dtype=object)

In [ ]:
# #Replace financial stage with 'Other'
# replace_stage = list(val_counts[val_counts <= 1.0].index)

# #Replace in df
# for values in replace_stage:
#     companies_df['Valuation ($B)']=companies_df['Valuation ($B)'].replace(values, "Other")
    
# #Verify binning was successful 
# companies_df["Valuation ($B)"].value_counts()

In [19]:
#Create a OneHotEncoder instance
enc = OneHotEncoder(sparse = False)

#Fit and transform the OneHotEncoder using the categorical variable list 
encode_df = pd.DataFrame(enc.fit_transform(companies_df[companies_cat]))

#Add the encoded variable names to the df
encode_df.columns = enc.get_feature_names_out(companies_cat)
encode_df.head()

,Company_1047 Games,Company_1KMXC,Company_1Password,Company_4Paradigm,Company_56PINGTAI,Company_58 Daojia,Company_6Sense,Company_ABL Space Systems,Company_AIWAYS,Company_ASAPP,...,Deal Terms_6,Deal Terms_7,Deal Terms_8,Deal Terms_9,Deal Terms_None,Portfolio Exits_1,Portfolio Exits_2,Portfolio Exits_3,Portfolio Exits_5,Portfolio Exits_None
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
